In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
train_df = pd.read_csv('./dataset/train/train_revised.csv')
test_df = pd.read_csv('./dataset/test/test_data.csv')

In [3]:
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

In [4]:
train_df_copy.head(3)

,Unnamed: 0,id,sentence,subject_entity,object_entity,label,source
0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree


In [5]:
test_df_copy.head(3)

,id,sentence,subject_entity,object_entity,label,source
0,0,지난 15일 MBC '탐사기획 스트레이트'가 이 사실을 보도했다.,"{'word': 'MBC', 'start_idx': 7, 'end_idx': 9, ...","{'word': '탐사기획 스트레이트', 'start_idx': 12, 'end_i...",100,wikitree
1,1,사랑스러운 ‘프린세스 프링’의 이름은 봄의 공주님: Princess(s)Pring이...,"{'word': '프린세스 프링', 'start_idx': 7, 'end_idx':...","{'word': '공주', 'start_idx': 84, 'end_idx': 85,...",100,wikipedia
2,2,"한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 1995년 6.25 특집극으...","{'word': '경찰', 'start_idx': 121, 'end_idx': 12...","{'word': '1995년', 'start_idx': 31, 'end_idx': ...",100,wikipedia


In [6]:
# 특수문자 있는 것 찾기
def find_sc(df,find):
    for i in range(len(df)):
        if find in df['sentence'][i]:
            print(df['sentence'][i])
            
def find_entity(df,find,entity):
    for i in range(len(df)):
        try:
            if find in df[entity][i].split('\', \'start_idx')[0].split('word\': \'')[1]:
                print(find,i,": ",df[entity][i])
        except:
            try:
                if find in df[entity][i].split('\", \'start_idx')[0].split('word\': \"')[1]:
                    print(find,i,": ",df[entity][i])
            except:
                print('!!error!!!!!!!',find,i,": ",df[entity][i])

In [7]:
# 문장 변경
def change_sc(df,find,new):
    for i in range(len(df)):
        if find in df['sentence'][i]:
            df['sentence'][i] = df['sentence'][i].replace(find,new)

In [8]:
# 토큰 변경
def change_entity(df,find,new,entity):
    for i in range(len(df)):
        if find in df[entity][i]:
            df[entity][i] = df[entity][i].replace(find,new)

In [9]:
# start_idx, end_idx이 잘 매치되었는지 체크
def check_idx(df,entity):
    idx_list =[]
    for i in range(len(df)):
        start_idx = int(df[entity][i].split(', \'end_idx')[0].split('start_idx\': ')[-1])
        end_idx = int(df[entity][i].split(', \'end_idx\': ')[1].split(', \'type')[0])
        try:
            word = df[entity][i].split('\', \'start_idx')[0].split('word\': \'')[1]
        except:
            try:
                word = df[entity][i].split('\", \'start_idx')[0].split('word\': \"')[1]
            except:
                print('!!error!!!!!!!',df['sentence'][i],i,": ",df[entity][i])
                
        if df['sentence'][i][start_idx:end_idx+1] != word:
            print(f"{i}: {df['id'][i]}, {df['sentence'][i][start_idx:end_idx+1]}, {word}")
            idx_list.append(i)
    return idx_list

In [10]:
# type
test_df_copy['object_entity'][35][-5:-2]
test_df_copy['subject_entity'][1]

"{'word': '프린세스 프링', 'start_idx': 7, 'end_idx': 13, 'type': 'PER'}"

In [11]:
# idx 매치 안되었을 때 변경하기
def change_idx(df,entity,idx_list):
    for i in idx_list:
        try:
            word = df[entity][i].split('\', \'start_idx')[0].split('word\': \'')[1]
        except:
            try:
                word = df[entity][i].split('\", \'start_idx')[0].split('word\': \"')[1]
            except:
                print('!!error!!!!!!!',df['sentence'][i],i,": ",df[entity][i])
        sentence = df['sentence'][i]
        type_entity = df[entity][i][-5:-2]
        try:
            start_idx = sentence.index(word) # 가장 처음에 나오는 위치로 변경합니다 (수정해야할수도...)
            end_idx = start_idx + len(word) -1
            df[entity][i] = f"{{'word': '{word}', 'start_idx': {start_idx}, 'end_idx': {end_idx}, 'type': '{type_entity}'}}"
        except ValueError:
            print('찾고자 하는 값 없음: ',word, i)

## (주) (사) ㈔ (재)  ->㈜ 로 통일
가정: 기업 관련된 class에 영향을 미칠 것으로 예상
klue/roberta-large 기준 ㈜ 는 토큰에 있으나 ㈔가 붙으면 UNK로 취급함
둘을 구별하는 class가 없어서 통일 결정

In [12]:
change_sc(train_df_copy,'(주)','㈜')
change_sc(train_df_copy,'(사)','㈜')
change_sc(train_df_copy,'㈔','㈜')
change_sc(train_df_copy,'(재)','㈜')

change_entity(train_df_copy,'(주)','㈜','subject_entity')
change_entity(train_df_copy,'(사)','㈜','subject_entity')
change_entity(train_df_copy,'㈔','㈜','subject_entity')
change_entity(train_df_copy,'(재)','㈜','subject_entity')

change_entity(train_df_copy,'(주)','㈜','object_entity')
change_entity(train_df_copy,'(사)','㈜','object_entity')
change_entity(train_df_copy,'㈔','㈜','object_entity')
change_entity(train_df_copy,'(재)','㈜','object_entity')

In [13]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_obj_list = check_idx(train_df_copy,'object_entity')

3: 3, 다이소(대, 아성다이소
608: 610, 대학교 총, 부산대학교
2018: 2024, 교통위원회(위, 국토교통위원회
2824: 2831, 비엔날레 등, 광주비엔날레
2968: 2975, EH)로, CJEH
3176: 3183, 헌 경, 전우헌
3950: 3957, 문화방송의 , 목포문화방송
4243: 4252, 수한 , 이투스
4668: 4677, 정보화진흥원 등, 한국정보화진흥원
5233: 5243, 정공㈜의, 현대정공
6690: 6704, 대학교 경, 동국대학교
6951: 6966, 농수산식품유통공사, , 한국농수산식품유통공사
6973: 6988, 군교육발전위원회(이, 고흥군교육발전위원회
7075: 7090, 트랜스 대, 네오트랜스
8672: 8689, 가스기술공사, , 한국가스기술공사
9247: 9265, 현대로템㈜는 , 현대로템㈜
10049: 10070, 고속이 , 광주고속
11244: 11267, ㈜마인버스를 , ㈜마인버스
11933: 11956, 정원정, 산림청
11948: 11971, 준 한, 박일준
12527: 12551, 시교육재단(이, 포천시교육재단
12633: 12657, 산업안전보건공단에서, 한국산업안전보건공단
14118: 14145, 과학기술원 총, 광주과학기술원
14856: 14883, 교통(이, 광양교통
15595: 15624, 플렉스는 , MJ플렉스
16144: 16174, 소(대, 다이소
16644: 16674, E미래에셋증권㈜으로, E미래에셋증권㈜
16824: 16854, 시도 , 여수시
17481: 17512, 기업기술정보진흥원, , 중소기업기술정보진흥원
17501: 17532, 군지부, 양평군
18071: 18102, 소프트의 , 엔씨소프트
19083: 19115, 군지부, 양평군
19820: 19853, 다이소(대, 아성다이소
19943: 19976, 소비자원이 , 한국소비자원
20054: 20087, 무역투자진흥공사(사, 대한무역투자진흥공사
20254: 20287, 북도무형, 전라북도
20306: 20339, 

In [14]:
change_idx(train_df_copy,'subject_entity',train_sub_list)
change_idx(train_df_copy,'object_entity',train_obj_list)

In [15]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_sub_list = check_idx(train_df_copy,'object_entity')

------------


In [16]:
change_sc(test_df_copy,'(주)','㈜')
change_sc(test_df_copy,'(사)','㈜')
change_sc(test_df_copy,'㈔','㈜')

change_entity(test_df_copy,'(주)','㈜','subject_entity')
change_entity(test_df_copy,'(사)','㈜','subject_entity')
change_entity(test_df_copy,'㈔','㈜','subject_entity')

change_entity(test_df_copy,'(주)','㈜','object_entity')
change_entity(test_df_copy,'(사)','㈜','object_entity')
change_entity(test_df_copy,'㈔','㈜','object_entity')

In [17]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_obj_list = check_idx(test_df_copy,'object_entity')

295: 295, 원(원장 , 광양문화원
696: 696, ㈜인천계양모범운전자회, , ㈜인천계양모범운전자회
1159: 1159,  방, 정부
1181: 1181, 준)와, 박일준
1704: 1704, 선 의, 홍귀선
2420: 2420, 철도㈜로, 경기철도
2898: 2898, ㈜광주민주화운동기념사업회 이, ㈜광주민주화운동기념사업회
3033: 3033, 석)은, 강제석
3235: 3235, Kcompany)를, YJKcompany
3477: 3477, ㈜빛남영화공동체(이, ㈜빛남영화공동체
3491: 3491, 시립예술단 정, 파주시립예술단
3686: 3686, 글로벌 세아㈜와 , 글로벌 세아㈜
4122: 4122, ㈜한국드론산업협회는 , ㈜한국드론산업협회
4306: 4306, 규 광, 김재규
5533: 5533, 노인대학(학, 가산노인대학
5660: 5660, ㈜한국전통민화연구소는 , ㈜한국전통민화연구소
7474: 7474, ㈜장보고글로벌재단(이, ㈜장보고글로벌재단
7558: 7558, 한국동서발전㈜(사, 한국동서발전㈜
------------
295: 295, 장 박, 이사장
1159: 1159, 객의 , 관광객
1181: 1181, 는 3, 홍동표
1704: 1704, 부시 부, 의정부시
2027: 2027, 대병원을 , 전남대병원
2260: 2260, 19년 대, 2019년
2898: 2898,  5·, 이사장
3033: 3033, 본부, 전남
3235: 3235, 자가 , 김연자
3477: 3477, 호)에, 이장호
3491: 3491, 감독)가, 음악감독
4122: 4122, 산업, 드론
4306: 4306, 장, , 부회장
5660: 5660, 전통, 한국
6277: 6277, 카페 등, 은빛카페
7474: 7474, 룡)이, 김덕룡
7558: 7558,  소, 중구


In [18]:
change_idx(test_df_copy,'subject_entity',test_sub_list)
change_idx(test_df_copy,'object_entity',test_obj_list)

In [19]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_obj_list = check_idx(test_df_copy,'object_entity')

------------


## unk 관련 수정

In [20]:
ch_list1 = ['⟪', '⟫', '˹', '˼','«' ,'»','⌜','⌟'] # to "'"
ch_list2 = ['€', ' Ⓐ', 'Ⓐ', '(˘ ³˘)!!!', '£', '▴', '⁺', '®', '、', '±'] # del
ch_list3 = ['⸱', '･', '⋅', '·']

In [21]:
change_sc(train_df_copy,' ▵',',')
change_sc(train_df_copy,'↔',',')
change_sc(train_df_copy,'–','-')
change_sc(train_df_copy,"＇",'\'')
change_sc(train_df_copy,"′",'\'')
change_sc(train_df_copy,'~','-')
change_sc(train_df_copy,'～','-')
change_sc(train_df_copy,'∼','-')
change_sc(train_df_copy,'（','(')
change_sc(train_df_copy,'）',')')
change_sc(train_df_copy,'㎿','MW')
change_sc(train_df_copy,'㎞','km')          
change_sc(train_df_copy,'㎏','kg')  
change_sc(train_df_copy,'','')  
          
for ch in ch_list1:
    change_sc(train_df_copy,ch,'\'')
for ch in ch_list2:
    change_sc(train_df_copy,ch,'')
for ch in ch_list3:
    change_sc(train_df_copy,ch,'/')

In [22]:
change_entity(train_df_copy,' ▵',',','subject_entity')
change_entity(train_df_copy,'↔',',','subject_entity')
change_entity(train_df_copy,'–','-','subject_entity')
change_entity(train_df_copy,"＇",'\'','subject_entity')
change_entity(train_df_copy,"′",'\'','subject_entity')
change_entity(train_df_copy,'~','-','subject_entity')
change_entity(train_df_copy,'～','-','subject_entity')
change_entity(train_df_copy,'∼','-','subject_entity')
change_entity(train_df_copy,'（','(','subject_entity')
change_entity(train_df_copy,'）',')','subject_entity')
change_entity(train_df_copy,'㎿','MW','subject_entity')
change_entity(train_df_copy,'㎞','km','subject_entity')          
change_entity(train_df_copy,'㎏','kg','subject_entity')  
change_entity(train_df_copy,'','','subject_entity')  
          
for ch in ch_list1:
    change_entity(train_df_copy,ch,'\'','subject_entity')
for ch in ch_list2:
    change_entity(train_df_copy,ch,'','subject_entity')
for ch in ch_list3:
    change_entity(train_df_copy,ch,'/','subject_entity')

In [23]:
change_entity(train_df_copy,' ▵',',','object_entity')
change_entity(train_df_copy,'↔',',','object_entity')
change_entity(train_df_copy,'–','-','object_entity')
change_entity(train_df_copy,"＇",'\'','object_entity')
change_entity(train_df_copy,"′",'\'','object_entity')
change_entity(train_df_copy,'~','-','object_entity')
change_entity(train_df_copy,'～','-','object_entity')
change_entity(train_df_copy,'∼','-','object_entity')
change_entity(train_df_copy,'（','(','object_entity')
change_entity(train_df_copy,'）',')','object_entity')
change_entity(train_df_copy,'㎿','MW','object_entity')
change_entity(train_df_copy,'㎞','km','object_entity')          
change_entity(train_df_copy,'㎏','kg','object_entity')  
change_entity(train_df_copy,'','','object_entity')  
          
for ch in ch_list1:
    change_entity(train_df_copy,ch,'\'','object_entity')
for ch in ch_list2:
    change_entity(train_df_copy,ch,'','object_entity')
for ch in ch_list3:
    change_entity(train_df_copy,ch,'/','object_entity')

In [24]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_obj_list = check_idx(train_df_copy,'object_entity')

820: 823, 코 F. 후지오는 , 후지코 F. 후지오
1635: 1639, 겔 앙헬 모야 , 미겔 앙헬 모야
2710: 2717,  핫토리군이 , 닌자 핫토리군
5249: 5259, 스널 FC에, 아스널 FC
7645: 7660, 제 모리뉴의, 주제 모리뉴
8977: 8995, 에몽의 , 도라에몽
9065: 9083, 리뉴의, 모리뉴
10616: 10638, 청‧한국교통안전, 한국교통안전공단
11819: 11842, 초 마레스카를, 엔초 마레스카
16502: 16532, 중 국, 유재중
22163: 22198, 상주식회사 , 대상주식회사
29640: 29681, fL 볼프스부르크로, VfL 볼프스부르크
------------
787: 790, 017년, 2017
820: 823, 에몽의 , 도라에몽
1238: 1241, 에몽, , 도라에몽
1635: 1639, 키퍼가, 골키퍼
2710: 2717, 후지코 후지오의 , 후지코 후지오
5249: 5259, 리미어리그의, 프리미어리그
6220: 6233, 코 F. 후지오와 , 후지코 F. 후지오
7645: 7660, 르투갈인, 포르투갈
8977: 8995, 코 F. 후지오는 , 후지코 F. 후지오
9065: 9083, 르투갈인, 포르투갈
9105: 9123, 알 마드리드로, 레알 마드리드
10616: 10638, , 국토교, 국토교통부
11819: 11842, 드필더 , 미드필더
14847: 14874, 화가, 일러스트레이터이, 만화가, 일러스트레이터
16502: 16532,  수영구),, 부산 수영구
22163: 22198, 대홍 , 임대홍
29640: 29681, 데스리가의, 분데스리가


In [25]:
change_idx(train_df_copy,'subject_entity',train_sub_list)
change_idx(train_df_copy,'object_entity',train_obj_list)

In [26]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_sub_list = check_idx(train_df_copy,'object_entity')

------------


In [27]:
change_sc(test_df_copy,' ▵',',')
change_sc(test_df_copy,'↔',',')
change_sc(test_df_copy,'–','-')
change_sc(test_df_copy,"＇",'\'')
change_sc(test_df_copy,"′",'\'')
change_sc(test_df_copy,'~','-')
change_sc(test_df_copy,'～','-')
change_sc(test_df_copy,'∼','-')
change_sc(test_df_copy,'（','(')
change_sc(test_df_copy,'）',')')
change_sc(test_df_copy,'㎿','MW')
change_sc(test_df_copy,'㎞','km')          
change_sc(test_df_copy,'㎏','kg')  
change_sc(test_df_copy,'','')  
          
for ch in ch_list1:
    change_sc(test_df_copy,ch,'\'')
for ch in ch_list2:
    change_sc(test_df_copy,ch,'')
for ch in ch_list3:
    change_sc(test_df_copy,ch,'/')

In [28]:
change_entity(test_df_copy,' ▵',',','subject_entity')
change_entity(test_df_copy,'↔',',','subject_entity')
change_entity(test_df_copy,'–','-','subject_entity')
change_entity(test_df_copy,"＇",'\'','subject_entity')
change_entity(test_df_copy,"′",'\'','subject_entity')
change_entity(test_df_copy,'~','-','subject_entity')
change_entity(test_df_copy,'～','-','subject_entity')
change_entity(test_df_copy,'∼','-','subject_entity')
change_entity(test_df_copy,'（','(','subject_entity')
change_entity(test_df_copy,'）',')','subject_entity')
change_entity(test_df_copy,'㎿','MW','subject_entity')
change_entity(test_df_copy,'㎞','km','subject_entity')          
change_entity(test_df_copy,'㎏','kg','subject_entity')  
change_entity(test_df_copy,'','','subject_entity')  
          
for ch in ch_list1:
    change_entity(test_df_copy,ch,'\'','subject_entity')
for ch in ch_list2:
    change_entity(test_df_copy,ch,'','subject_entity')
for ch in ch_list3:
    change_entity(test_df_copy,ch,'/','subject_entity')

In [29]:
change_entity(test_df_copy,' ▵',',','object_entity')
change_entity(test_df_copy,'↔',',','object_entity')
change_entity(test_df_copy,'–','-','object_entity')
change_entity(test_df_copy,"＇",'\'','object_entity')
change_entity(test_df_copy,"′",'\'','object_entity')
change_entity(test_df_copy,'~','-','object_entity')
change_entity(test_df_copy,'～','-','object_entity')
change_entity(test_df_copy,'∼','-','object_entity')
change_entity(test_df_copy,'（','(','object_entity')
change_entity(test_df_copy,'）',')','object_entity')
change_entity(test_df_copy,'㎿','MW','object_entity')
change_entity(test_df_copy,'㎞','km','object_entity')          
change_entity(test_df_copy,'㎏','kg','object_entity')  
change_entity(test_df_copy,'','','object_entity')  
          
for ch in ch_list1:
    change_entity(test_df_copy,ch,'\'','object_entity')
for ch in ch_list2:
    change_entity(test_df_copy,ch,'','object_entity')
for ch in ch_list3:
    change_entity(test_df_copy,ch,'/','object_entity')

In [30]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_obj_list = check_idx(test_df_copy,'object_entity')

2137: 2137,  모정, 모정이
4148: 4148,  대표,, 더 이음
7091: 7091, 터 시티와 4, 맨체스터 시티
7746: 7746, 정자치부 , 행정자치부
------------
2137: 2137, 10k, 10kg
4148: 4148, 왕기 , 이왕기


In [31]:
change_idx(test_df_copy,'subject_entity',test_sub_list)
change_idx(test_df_copy,'object_entity',test_obj_list)

In [32]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_sub_list = check_idx(test_df_copy,'object_entity')

------------


## 중복기호 제거
kg km는 위에있음..

In [33]:
same = ['：','‘','’','“','”','？','／','％']
change_same = [':','\'','\'','\"','\"','?','/','%']

In [34]:
for i in range(len(same)):
    change_sc(train_df_copy,same[i],change_same[i])  


In [35]:
for i in range(len(same)):
    change_entity(train_df_copy,same[i],change_same[i],'subject_entity')  


In [36]:
for i in range(len(same)):
    change_entity(train_df_copy,same[i],change_same[i],'object_entity')  

In [37]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_obj_list = check_idx(train_df_copy,'object_entity')

------------


In [38]:
change_idx(train_df_copy,'subject_entity',train_sub_list)
change_idx(train_df_copy,'object_entity',train_obj_list)

In [39]:
train_sub_list = check_idx(train_df_copy,'subject_entity')
print('------------')
train_sub_list = check_idx(train_df_copy,'object_entity')

------------


In [40]:
for i in range(len(same)):
    change_sc(test_df_copy,same[i],change_same[i])  


In [41]:
for i in range(len(same)):
    change_entity(test_df_copy,same[i],change_same[i],'subject_entity')  


In [42]:
for i in range(len(same)):
    change_entity(test_df_copy,same[i],change_same[i],'object_entity')  


In [43]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_obj_list = check_idx(test_df_copy,'object_entity')

------------


In [44]:
change_idx(test_df_copy,'subject_entity',test_sub_list)
change_idx(test_df_copy,'object_entity',test_obj_list)

In [45]:
test_sub_list = check_idx(test_df_copy,'subject_entity')
print('------------')
test_sub_list = check_idx(test_df_copy,'object_entity')

------------


In [46]:
sc_dict_all = {}
for i in range(len(train_df_copy)):
    string = train_df_copy['sentence'][i]
    for char in string:
        if not char.isalnum():
            try:
                sc_dict_all[char] += 1
            except:
                sc_dict_all[char] = 1
for i in range(len(test_df_copy)):
    string = test_df_copy['sentence'][i]
    for char in string:
        if not char.isalnum():
            try:
                sc_dict_all[char] += 1
            except:
                sc_dict_all[char] = 1

In [47]:
sc_dict_all

{'〈': 224,
 '〉': 230,
 ' ': 788013,
 '《': 1131,
 '》': 1122,
 '.': 44816,
 '·': 3626,
 '(': 21766,
 ')': 21740,
 "'": 13833,
 '㈜': 515,
 ':': 765,
 ',': 44570,
 '-': 6933,
 '"': 13907,
 '‧': 181,
 '▲': 362,
 '%': 1382,
 '/': 341,
 '&': 260,
 '!': 168,
 ';': 62,
 '=': 17,
 '→': 30,
 '△': 232,
 '*': 20,
 '｢': 11,
 '｣': 11,
 '★': 3,
 '>': 2,
 '․': 133,
 '「': 130,
 '」': 130,
 '・': 41,
 '•': 8,
 '☎': 28,
 '[': 398,
 ']': 391,
 '☆': 2,
 '`': 39,
 '※': 4,
 '|': 38,
 '『': 37,
 '』': 37,
 '。': 3,
 '，': 4,
 '㎡': 17,
 '́': 2,
 '_': 7,
 '∞': 3,
 '♥': 5,
 '{': 6,
 '}': 6,
 '°': 2,
 '℃': 2,
 '∙': 15,
 '♡': 2,
 '$': 13,
 '㎖': 1,
 '≪': 5,
 '≫': 5,
 '□': 1,
 '\u200e': 2,
 '☏': 4,
 '?': 1,
 '―': 5,
 '○': 5,
 '़': 2,
 'ी': 4,
 'ु': 4,
 '्': 4,
 'ा': 2,
 '×': 5,
 '—': 8,
 '₫': 1,
 '<': 1,
 '㎜': 1,
 '\uf09e': 1,
 '´': 1,
 '●': 1,
 '▶': 6}

In [54]:
train_df_copy.to_csv('./dataset/train/train_spc_char1.csv',header = True,index = False)

In [55]:
test_df_copy.to_csv('./dataset/test/test_spc_char1.csv',header = True,index = False)